# Rolling Ball Figure

In [1]:
import numpy as np
import pathlib
import meshio
import trimesh
import igl
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from tqdm import tqdm

In [2]:
matlab_colors = ['#0072BD', '#D95319', '#EDB120', '#7E2F8E', '#77AC30', '#4DBEEE', '#A2142F']

In [3]:
baseline_results_dir = pathlib.Path('../../results/slope/baseline/')
our_results_dir = pathlib.Path('../../results/slope/bc/')

In [4]:
def fix_normals(V, F):
    mesh = trimesh.Trimesh(V, F, process=False, validate=False)
    trimesh.repair.fix_normals(mesh)
    return mesh.faces


def load_mesh(path):
    mesh = meshio.read(path)

    V, I, J, _ = igl.remove_duplicate_vertices(
        mesh.points, np.array([], dtype=int), 1e-7)

    CV = []  # codim vertices
    E = []  # edges
    F = []  # triangles
    T = []  # tetrahedra
    for cells in mesh.cells:
        if cells.type == "triangle":
            F.append(J[cells.data])
        elif cells.type == "tetra":
            # F.append(fix_normals(V, igl.boundary_facets(J[cells.data])))
            T.append(J[cells.data])
        elif cells.type == "line":
            E.append(J[cells.data])
        elif cells.type == "vertex":
            CV.append(J[cells.data])
        else:
            raise Exception("Unsupported cell type: {}".format(cells.type))

    cells = []
    if T:
        cells.append(("tetra", np.vstack(T)))
    if F:
        cells.append(("triangle", np.vstack(F)))
    if E:
        cells.append(("line", np.vstack(E)))
    if CV:
        cells.append(("vertex", np.vstack(CV)))

    if "solution" in mesh.point_data:
        V += mesh.point_data["solution"][I]

    point_data = dict((k, v[I]) for k, v in mesh.point_data.items())
    if "E" in point_data:
        point_data["is_obstacle"] = (point_data["E"] == 0).flatten()
    else:
        point_data["is_obstacle"] = np.zeros((V.shape[0],), dtype=bool)

    # V[point_data["is_obstacle"]] *= [[1e3, 0, 1e3]]

    mesh = meshio.Mesh(points=V, cells=cells, point_data=point_data)

    return mesh

In [5]:
def compute_out_of_plane_friction_force(mesh, ord=1):
    assert(mesh.cells[1].type == "triangle")
    plane = mesh.points[mesh.cells[1].data[0]]
    n = np.cross(plane[1] - plane[0], plane[2] - plane[0])
    n /= np.linalg.norm(n)
    return np.linalg.norm([ff.dot(n) for ff in mesh.point_data["friction_forces"]], ord=ord)

In [6]:
def load_data(dir, nsteps=200):
    out_of_plane_friction_force = np.zeros(nsteps + 1)
    z_friction_force = np.zeros(nsteps + 1)
    contact_force = np.zeros(nsteps + 1)
    for i in tqdm(range(nsteps + 1)):
        mesh = load_mesh(dir / f"step_{i}.vtu")
        out_of_plane_friction_force[i] = compute_out_of_plane_friction_force(mesh, ord=1)
        z_friction_force[i] = np.linalg.norm(mesh.point_data["friction_forces"][:, 2], ord=1)
        contact_force[i] = np.linalg.norm(mesh.point_data["contact_forces"], ord=1)
    return out_of_plane_friction_force, z_friction_force, contact_force

In [7]:
our_ferr, _, _ = load_data(our_results_dir)
# baseline_ferr, _, _ = load_data(baseline_results_dir)

  0%|          | 0/201 [00:00<?, ?it/s]

ReadError: File ../../results/slope/bc/step_0.vtu not found.

In [ ]:
# Constants factors
margin_inches = 1/16
ppi = 300
default_ppi = 96
scale = ppi / default_ppi

width_inches = 0.89 * 3.5
height_inches = 1/4 * 3.5
width = (width_inches - margin_inches) * ppi
height = (height_inches - margin_inches) * ppi

font_size = scale * 8.25
margin = margin_inches * ppi

paper_style = dict(
    width=width,
    height=height,
    title=dict(
        x=0.5,
        pad=dict(b=0,l=0,r=0,t=0),
    ),
    xaxis=dict(
        ticks="inside",
        color="black",
        linecolor="black",
    ),
    xaxis2=dict(
        ticks="inside",
        color="black",
        linecolor="black",
    ),
    yaxis=dict(
        ticks="inside",
        color="black",
        linecolor="black",
    ),
    yaxis2=dict(
        ticks="inside",
        color="black",
        linecolor="black",
    ),
    template="simple_white",
    font=dict(
        family="Linux Libertine",
        size=font_size,
        color="black"
    ),
    legend=dict(
        font=dict(
            family="Linux Libertine",
            size=font_size
        ),
        orientation="h",
        yanchor="top",
        y=1.05,
        xanchor="center",
        x=0.5,
        bgcolor='rgba(0,0,0,0)'
    ),
    margin=dict(l=margin, r=margin, t=margin, b=margin)
)

In [ ]:
# fig = go.Figure()
line_width=1

tend = 5
nsteps = our_ferr.size
t = np.linspace(0, tend, nsteps)
last_step = int(3 / tend * nsteps) + 1

fig = make_subplots(
    rows=1,
    cols=1, 
    shared_xaxes=True, 
    shared_yaxes=True,
    # vertical_spacing = 0.04, 
    # y_title="$\|F \cdot n\|_1$"
)

trace = go.Scatter(
    x=t[:last_step],
    y=baseline_ferr[:last_step],
    name="Baseline (352.4 s)",
    line=dict(
        width=line_width,
        color=matlab_colors[0]
    )
)
fig.add_trace(trace, row=1, col=1)
# fig.add_trace(trace, row=2, col=1)

trace = go.Scatter(
    x=t[:last_step],
    y=our_ferr[:last_step],
    name="Ours (47.0 s)",
    line=dict(
        width=line_width,
        color=matlab_colors[1]
    )
)
fig.add_trace(trace, row=1, col=1)
# fig.add_trace(trace, row=2, col=1)

fig.update_layout(**paper_style)
fig.update_layout(
    xaxis=dict(
        # visible=False,
        # ticks=""
        title="time (s)"
    ),
    xaxis2=dict( 
        title="time (s)"
    ), 
    yaxis=dict(
        # title="$\|F \cdot n\|_1$",
        range=[0, 3.5e-16],
        zeroline=False,
        # tickformat='.0',
        # dtick=1,
        # tick0=-4,
        # type="log"
    ),
    yaxis2=dict(
        # range=[0, 1e-15],
        # dtick=2e-16,
        tickformat='.0',
    ),
)

# for i, trace in enumerate(fig['data']): 
#     if i % 2 == 1: 
#         trace['showlegend'] = False

# Write figure with dimensions in inches
pio.write_image(
    fig, 'rolling-ball-friction.pdf',
    width=width,
    height=height,
    scale=1/scale)
fig.show()

NameError: name 'baseline_ferr' is not defined

In [ ]:
import json
with open(our_results_dir / "sim.json") as f:
    sim = json.load(f)
print("{:.1f} s".format(sum(info["info"]["total_time"] for info in sim["solver_info"])))

47.0 s


In [ ]:
with open(baseline_results_dir / "sim.json") as f:
    sim = json.load(f)
print("{:.1f} s".format(sum(info["info"]["total_time"] for info in sim["solver_info"])))

352.4 s
